In [1]:
import torch
import wandb
from tqdm.auto import tqdm
import pytorch_lightning as pl

import eugene as eu
import eugene.train
import eugene.models
from eugene import settings

import seqdata as sd
import motifdata as md

from copy import deepcopy

In [2]:
settings.dataset_dir = "/cellar/users/aklie/data/eugene/revision/jores21"
settings.output_dir = "/cellar/users/dlaub/projects/ML4GLand/EUGENe_paper/output/jores21"
settings.logging_dir = "/cellar/users/dlaub/projects/ML4GLand/EUGENe_paper/logs/jores21"
settings.config_dir = "/cellar/users/dlaub/projects/ML4GLand/EUGENe_paper/configs/jores21"
wandb.login()

wandb: Currently logged in as: dlaub. Use `wandb login --relogin` to force relogin


True

In [3]:
sdata = (
    sd.open_zarr('/cellar/users/aklie/data/eugene/revision/jores21/jores21_leaf_train.zarr')
    .rename({"enrichment": 'target'})
)
sdata

<xarray.Dataset>
Dimensions:     (_sequence: 65004, length: 170, _ohe: 4)
Dimensions without coordinates: _sequence, length, _ohe
Data variables: (12/19)
    GC          (_sequence) float64 dask.array<chunksize=(32502,), meta=np.ndarray>
    UTR         (_sequence) float64 dask.array<chunksize=(32502,), meta=np.ndarray>
    barcodes    (_sequence) float64 dask.array<chunksize=(32502,), meta=np.ndarray>
    batch       (_sequence) <U5 dask.array<chunksize=(16251,), meta=np.ndarray>
    chromosome  (_sequence) object dask.array<chunksize=(32502,), meta=np.ndarray>
    end         (_sequence) object dask.array<chunksize=(32502,), meta=np.ndarray>
    ...          ...
    set         (_sequence) object dask.array<chunksize=(32502,), meta=np.ndarray>
    sp          (_sequence) int64 dask.array<chunksize=(32502,), meta=np.ndarray>
    start       (_sequence) object dask.array<chunksize=(32502,), meta=np.ndarray>
    strand      (_sequence) object dask.array<chunksize=(32502,), meta=np.ndarray>
    train_val   (_sequence) bool dask.array<chunksize=(65004,), meta=np.ndarray>
    type        (_sequence) object dask.array<chunksize=(32502,), meta=np.ndarray>
Attributes:
    max_jitter:  0

In [4]:
sdata[['ohe_seq', 'target']].load();

In [5]:
train_sdata = sdata.sel(_sequence=(sdata.train_val == True).compute())

transforms = {
    "ohe_seq": lambda x: x.swapaxes(1, 2)
}

dl = sd.get_torch_dataloader(
    train_sdata,
    sample_dims='_sequence',
    variables=['ohe_seq', 'target'],
    transforms=transforms,
    batch_size=1024,
    pin_memory=True,
    drop_last=False,
)

In [ ]:
# core_promoter_elements = md.read_meme("/cellar/users/aklie/data/eugene/revision/jores21/CPEs.meme")
# tf_clusters = md.read_meme("/cellar/users/aklie/data/eugene/revision/jores21/TF-clusters.meme")
# all_motifs = deepcopy(core_promoter_elements)
# for motif in tf_clusters:
#     all_motifs.add_motif(motif)

### PyTorch

In [22]:
model = eu.models.load_config(
    config_path='/cellar/users/aklie/projects/ML4GLand/EUGENe_paper/configs/jores21/hybrid.yaml',
    seed=0
).cuda()

eu.models.init_weights(model, initializer="kaiming_normal")

# eu.models.init_motif_weights(
#     model=model,
#     layer_name="arch.conv1d_tower.layers.0",
#     list_index=None,
#     initializer="xavier_uniform",
#     motifs=all_motifs,
#     convert_to_pwm=False,
#     divide_by_bg=True,
#     motif_align="left",
#     kernel_align="left"
# )

[rank: 0] Global seed set to 0


In [23]:
def train(model, dloader, optim, loss_fn):
    model.train()
    for batch in tqdm(dloader, position=1, leave=False):
        pred = model(batch['ohe_seq'].cuda())
        loss = loss_fn(pred.squeeze(), batch['target'].cuda())
        loss.backward()
        optim.step()
        optim.zero_grad()

In [24]:
epochs = 25
optim = torch.optim.Adam(model.parameters())
with wandb.init(project='EUGENe GPU Utilization', name='Native PyTorch', tags=['Jores21']):
    for _ in tqdm(range(epochs), position=0):
        train(model, dl, optim, model.loss_fxn)

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/58 [00:00<?, ?it/s]

  0%|          | 0/58 [00:00<?, ?it/s]

  0%|          | 0/58 [00:00<?, ?it/s]

  0%|          | 0/58 [00:00<?, ?it/s]

...

  0%|          | 0/58 [00:00<?, ?it/s]

  0%|          | 0/58 [00:00<?, ?it/s]

  0%|          | 0/58 [00:00<?, ?it/s]

  0%|          | 0/58 [00:00<?, ?it/s]

wandb: ERROR Control-C detected -- Run data was not synced


### Lightning

In [10]:
model = eu.models.load_config(
    config_path='/cellar/users/aklie/projects/ML4GLand/EUGENe_paper/configs/jores21/hybrid.yaml',
    seed=0
).cuda()

eu.models.init_weights(model, initializer="kaiming_normal")

# eu.models.init_motif_weights(
#     model=model,
#     layer_name="arch.conv1d_tower.layers.0",
#     list_index=None,
#     initializer="xavier_uniform",
#     motifs=all_motifs,
#     convert_to_pwm=False,
#     divide_by_bg=True,
#     motif_align="left",
#     kernel_align="left"
# )

model.scheduler = None

[rank: 0] Global seed set to 0


In [11]:
trainer = pl.Trainer(max_epochs=25, logger=False)

/cellar/users/dlaub/miniconda3/envs/eugene-dev/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3.9 /cellar/users/dlaub/miniconda3/envs/eugene-dev/li ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [12]:
with wandb.init(project='EUGENe GPU Utilization', name='PL no val', tags=['Jores21']):
    trainer.fit(model, dl)

/cellar/users/dlaub/miniconda3/envs/eugene-dev/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3.9 /cellar/users/dlaub/miniconda3/envs/eugene-dev/li ...
  rank_zero_warn(
/cellar/users/dlaub/miniconda3/envs/eugene-dev/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
/cellar/users/dlaub/miniconda3/envs/eugene-dev/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:615: UserWarning: Checkpoint directory /cellar/users/dlaub/projects/ML4GLand/EUGENe_paper/notebooks/gpu_utilization_analysis/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and 

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=25` reached.


### EUGENe

In [6]:
model = eu.models.load_config(
    config_path='/cellar/users/aklie/projects/ML4GLand/EUGENe_paper/configs/jores21/hybrid.yaml',
    seed=0
).cuda()

eu.models.init_weights(model, initializer="kaiming_normal")

# eu.models.init_motif_weights(
#     model=model,
#     layer_name="arch.conv1d_tower.layers.0",
#     list_index=None,
#     initializer="xavier_uniform",
#     motifs=all_motifs,
#     convert_to_pwm=False,
#     divide_by_bg=True,
#     motif_align="left",
#     kernel_align="left"
# )

model.scheduler = None

[rank: 0] Global seed set to 0


In [7]:
# Fit the model
with wandb.init(project='EUGENe GPU Utilization', name='EUGENe', tags=['Jores21']):
    eu.train.fit_sequence_module(
        model,
        sdata,
        seq_key="ohe_seq",
        target_keys=['target'],
        in_memory=True,
        train_key="train_val",
        epochs=25,
        batch_size=1024,
        drop_last=False,
        early_stopping_metric=None,
        model_checkpoint_monitor=None,
        transforms=transforms
    )

Dropping 0 sequences with NaN targets.
Loading ohe_seq and ['target'] into memory
No seed set


/cellar/users/dlaub/miniconda3/envs/eugene-dev/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3.9 /cellar/users/dlaub/miniconda3/envs/eugene-dev/li ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/cellar/users/dlaub/miniconda3/envs/eugene-dev/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3.9 /cellar/users/dlaub/miniconda3/envs/eugene-dev/li ...
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

 

Sanity Checking: 0it [00:00, ?it/s]

/cellar/users/dlaub/miniconda3/envs/eugene-dev/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/cellar/users/dlaub/miniconda3/envs/eugene-dev/lib/python3.9/site-packages/torch/nn/modules/conv.py:309: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1682343997789/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,
/cellar/users/dlaub/miniconda3/envs/eugene-dev/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottl

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=25` reached.
